In [1]:
!pip install PyWavelets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 29.1 MB/s eta 0:00:00


In [3]:
!pip install lpips


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [4]:
import os
import lpips
import torch
import pandas as pd

import pathlib


In [5]:
from PIL import Image
from torchvision import transforms
from pathlib import Path

In [25]:
path_original = Path("/content/input_images")

path_input_dct = Path("/content/Test_dct")

path_dct = Path("/content/input_dct")

path_mdct = Path("/content/output_images")

In [26]:
# Initialize LPIPS model
lpips_model = lpips.LPIPS(net='alex')  # Options: alex, vgg, squeeze

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth


In [27]:
# Define image transformation
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize for uniformity
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [28]:
# Function to calculate LPIPS for a set of images
def calculate_lpips(original_path, compressed_path):
    total_lpips = 0.0
    image_count = 0
    data = []

    for filename in os.listdir(original_path):
        original_image_path = os.path.join(original_path, filename)
        compressed_image_path = os.path.join(compressed_path, filename)

        # Check if corresponding compressed image exists and skip directories
        if not os.path.isfile(original_image_path) or not os.path.isfile(compressed_image_path):
            continue

        # Load images
        original_image = Image.open(original_image_path).convert('RGB')
        compressed_image = Image.open(compressed_image_path).convert('RGB')

        # Transform images
        original_tensor = transform(original_image).unsqueeze(0)
        compressed_tensor = transform(compressed_image).unsqueeze(0)

        # Calculate LPIPS
        lpips_value = lpips_model(original_tensor, compressed_tensor)
        lpips_value_item = lpips_value.item()
        total_lpips += lpips_value_item
        image_count += 1

        #print(f"LPIPS for {filename}: {lpips_value.item()}")

        # Store result
        data.append({'File Name':filename, 'LPIPS Value':lpips_value_item})

    # Create DataFrame
    df = pd.DataFrame(data)
    return df

    # Calculate average LPIPS
    #avg_lpips = total_lpips / image_count if image_count > 0 else 0
    #return avg_lpips

In [29]:
# Calculate LPIPS for color images
lpips_mdct = calculate_lpips(path_original, path_mdct)
#print(f"Average LPIPS for color images: {lpips_color}")
print("LPIPS Scores for MDCT compressed Images with Original Images:")
print(lpips_mdct)
#lpips_color.to_csv('/home/jupyter-64365/VC project/color_lpips_results.csv', index=False)

LPIPS Scores for MDCT compressed Images with Original Images:
   File Name  LPIPS Value
0    13.jpeg     0.517091
1     9.jpeg     0.331022
2    17.jpeg     0.365689
3    24.jpeg     0.402504
4     8.jpeg     0.309440
5    23.jpeg     0.665594
6    15.jpeg     0.430111
7    11.jpeg     0.389284
8    22.jpeg     0.535063
9    20.jpeg     0.294989
10    5.jpeg     0.509890
11    6.jpeg     0.377356
12   16.jpeg     0.314706
13   18.jpeg     0.541192
14   14.jpeg     0.545919
15    3.jpeg     0.496875
16    1.jpeg     0.352045
17   21.jpeg     0.381031
18   12.jpeg     0.393769
19    2.jpeg     0.648277
20   19.jpeg     0.327316
21    7.jpeg     0.515413
22   10.jpeg     0.366413
23    4.jpeg     0.576798


In [30]:
# Calculate LPIPS for gray images
lpips_dct = calculate_lpips(path_input_dct, path_dct)
#print(f"Average LPIPS for gray images: {lpips_gray}")
print("LPIPS Scores for DCT Compresssed Images with Original Images:")
print(lpips_dct)
#lpips_gray.to_csv('/home/jupyter-64365/VC project/gray_lpips_results.csv', index=False)

LPIPS Scores for DCT Compresssed Images with Original Images:
Empty DataFrame
Columns: []
Index: []


In [ ]:

# Function to calculate LPIPS for a set of images
'''def calculate_lpips(original_path, compressed_path):
    total_lpips = 0.0
    image_count = 0

    for filename in os.listdir(original_path):
        original_image_path = os.path.join(original_path, filename)
        compressed_image_path = os.path.join(compressed_path, filename)

        # Check if corresponding compressed image exists
        if not os.path.exists(compressed_image_path):
            continue

        # Load images
        original_image = Image.open(original_image_path).convert('RGB')
        compressed_image = Image.open(compressed_image_path).convert('RGB')

        # Transform images
        original_tensor = transform(original_image).unsqueeze(0)
        compressed_tensor = transform(compressed_image).unsqueeze(0)

        # Calculate LPIPS
        lpips_value = lpips_model(original_tensor, compressed_tensor)
        total_lpips += lpips_value.item()
        image_count += 1

        print(f"LPIPS for {filename}: {lpips_value.item()}")

    # Calculate average LPIPS
    avg_lpips = total_lpips / image_count if image_count > 0 else 0
    return avg_lpips